# Classificação tabular

### 🌾 **Classificação de Sementes de Trigo - Explicação Inicial**  

Este conjunto de dados contém **medidas físicas de sementes de trigo** e tem como objetivo a **classificação da espécie** com base nessas características. Cada linha representa uma semente e inclui as seguintes informações:  

🔹 **Características das sementes:**  
- **Área**: Tamanho da superfície da semente.  
- **Perímetro**: Medida do contorno da semente.  
- **Compacidade**: Grau de proximidade entre as partes da semente.  
- **Comprimento**: Tamanho longitudinal da semente.  
- **Largura**: Largura máxima da semente.  
- **Assimetria**: Diferença na forma da semente em relação a um eixo.  
- **Comprimento do sulco**: Medida do sulco central da semente.  

🔹 **Classe de saída:**  
- **Espécie**: Representa o tipo de semente de trigo (0, 1 ou 2), indicando a qual variedade ela pertence.  

O objetivo do problema é utilizar essas medidas para treinar um modelo de **classificação supervisionada**, permitindo identificar corretamente a espécie de uma nova semente com base nas suas características. 🚀🌾

In [ ]:
import pandas as pd

In [ ]:
dados = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/Primeiros_passos_pytorch/main/sementes.csv')
dados.head()

In [ ]:
dados['Espécie'].unique()

In [ ]:
X = dados.drop(['Espécie'],axis=1).values
y = dados['Espécie'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_treino,X_teste,y_treino,y_teste = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import StandardScaler

## Normalização dos dados

https://pytorch.org/docs/stable/tensors.html

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

## Criando DataLoaders

## Definição da Rede Neural

https://pytorch.org/docs/stable/generated/torch.nn.Module.html

### 📌 **O que é `nn.Linear`?**
`nn.Linear(in_features, out_features)` é uma camada totalmente conectada (ou densa), que realiza uma transformação linear dos dados de entrada:


$$ Y = XW^T + b $$


onde:
- \( X \) é o tensor de entrada
- \( W \) são os pesos da camada
- \( b \) é o bias (termo de deslocamento)
- \( Y \) é a saída transformada

Essa camada aprende \( W \) e \( b \) durante o treinamento para mapear a entrada na saída desejada.

---

### 🔹 **Explicação da Arquitetura**
1. **Camada oculta (`self.fc1`)**  
   ```python
   self.fc1 = nn.Linear(input_size, hidden_size)
   ```
   - Converte a entrada (`input_size`) para um espaço de representação de dimensão `hidden_size`.
   - Aprenderá pesos $ W_1 $ e bias $ b_1$ para essa transformação.

2. **Ativação ReLU (`self.relu`)**  
   ```python
   self.relu = nn.ReLU()
   ```
   - Aplica a função de ativação ReLU:
     
     $$ f(x) = \max(0, x) $$
     
   - Introduz não-linearidade, permitindo que a rede aprenda padrões mais complexos.

3. **Camada de saída (`self.fc2`)**  
   ```python
   self.fc2 = nn.Linear(hidden_size, output_size)
   ```
   - Transforma a saída da camada oculta (`hidden_size`) no número final de classes (`output_size`).
   - A saída ainda não está normalizada (logits), então usamos `CrossEntropyLoss` para lidar com isso.

---

### 🔥 **Fluxo de Dados (`forward`)**
```python
def forward(self, x):
    x = self.fc1(x)   # Transformação linear
    x = self.relu(x)  # Ativação ReLU
    x = self.fc2(x)   # Segunda transformação linear
    return x          # Saída (logits)
```
- A entrada passa pela primeira camada densa (`fc1`).
- Depois, aplicamos a ativação `ReLU`, introduzindo não-linearidade.
- Finalmente, a saída passa pela camada `fc2`, que gera 3 valores (um para cada classe).

💡 **Observação:**  
A saída do modelo são **logits** (valores não normalizados). Para obter probabilidades, aplicaríamos `nn.Softmax(dim=1)`, mas `CrossEntropyLoss` já faz isso internamente.

---

### 🚀 **Resumo**
- `nn.Linear` realiza transformações lineares de entrada para saída.
- `ReLU` adiciona não-linearidade para melhorar a capacidade de aprendizado.
- O modelo é uma rede neural simples com **uma camada oculta** e **três neurônios de saída** (um para cada classe).

## Definição de hiperparâmetros

### 📌 **Regras práticas para definir o número de neurônios na camada oculta**
Como o número de features de entrada é **7**, podemos usar algumas heurísticas comuns:

1️⃣ **Regra da média geométrica:**  
$$ \text{neurônios na camada oculta} = \sqrt{\text{neurônios de entrada} \times \text{neurônios de saída}}
$$
Aplicando ao nosso caso:
$$
h = \sqrt{7 \times 3} = \sqrt{21} \approx 4 \text{ ou } 5
$$

2️⃣ **Regra do dobro do número de entradas:**  
$$
h = 2 \times \text{número de features} = 2 \times 7 = 14
$$
Se o problema for complexo, pode ser útil começar com mais neurônios.

3️⃣ **Regra do "funil" (entre entrada e saída):**  
- A camada oculta geralmente tem um número intermediário de neurônios entre a entrada e a saída.
- Um valor comum seria algo entre $  (7+3)/2 = 5;  2 \times 7 = 14  $.

---

### 🚀 **Escolha prática**
Se for um problema simples, **5 a 10 neurônios** na camada oculta pode ser um bom começo.  
Se for um problema mais complexo, com padrões difíceis de aprender, **10 a 14 neurônios** pode ser melhor.  

#### **Aplicando no código:**
```python
hidden_size = 5  # Ou 7, 10, 14, dependendo dos testes
model = Classifier(input_size=7, hidden_size=hidden_size, output_size=3)
```

---

### 🔥 **Melhor estratégia: Experimentação!**
O melhor número de neurônios depende da complexidade do problema e dos dados. O ideal é testar diferentes configurações e observar a **performance no conjunto de validação**, usando técnicas como **cross-validation** ou **grid search**.

## Inicialização do modelo

## Treinamento do modelo

### **1. Estrutura do loop de treinamento**
O treinamento do modelo é dividido em **épocas** (`epochs`) e **lotes (batches)**. Cada época representa uma iteração completa sobre o conjunto de dados de treinamento, enquanto o processamento em lotes divide o conjunto de dados em partes menores para melhorar a eficiência computacional e permitir o uso de gradientes estocásticos.

---

### **2. Passo a passo do código**

#### **(a) Loop de épocas**
```python
for epoch in range(epochs):
```
- Este loop externo controla o número total de épocas de treinamento. Cada época representa uma passagem completa pelo conjunto de dados de treinamento.
- `epochs` é o número total de vezes que o modelo verá o conjunto de dados.

#### **(b) Modo de treinamento**
```python
model.train()
```
- Coloca o modelo no **modo de treinamento**. Isso é importante porque algumas camadas (como `Dropout` ou `BatchNorm`) se comportam de forma diferente durante o treinamento e a inferência.
- No modo de treinamento, essas camadas ajustam seus parâmetros ou aplicam regularização.

#### **(c) Inicialização do total da perda**
```python
total_loss = 0
```
- Aqui, é inicializada uma variável para acumular a perda total ao longo de todos os lotes da época. Isso será usado para calcular a perda média no final da época.

---

### **3. Loop de lotes (batches)**
```python
for X_batch, y_batch in train_loader:
```
- Este loop interno itera sobre os dados de treinamento divididos em lotes (batches), que são fornecidos pelo `DataLoader` (`train_loader`).
- Cada `X_batch` contém um subconjunto dos dados de entrada, e `y_batch` contém os rótulos correspondentes.

#### **(a) Envio dos dados para o dispositivo**
```python
X_batch, y_batch = X_batch.to(device), y_batch.to(device)
```
- Move os dados de entrada (`X_batch`) e os rótulos (`y_batch`) para o dispositivo de computação configurado (CPU ou GPU). Isso é necessário para garantir que os dados e o modelo estejam no mesmo dispositivo durante o treinamento.

#### **(b) Zerar os gradientes acumulados**
```python
optimizer.zero_grad()
```
- Zera os gradientes acumulados nos parâmetros do modelo. No PyTorch, os gradientes são acumulados por padrão, então é necessário limpá-los antes de calcular os novos gradientes para o lote atual.

#### **(c) Forward pass**
```python
outputs = model(X_batch)
```
- Passa os dados de entrada (`X_batch`) pelo modelo, gerando as saídas (`outputs`), que geralmente são os logits (valores antes da aplicação de softmax).

#### **(d) Cálculo da perda**
```python
loss = criterion(outputs, y_batch)
```
- Calcula a perda entre as saídas previstas pelo modelo (`outputs`) e os rótulos verdadeiros (`y_batch`) usando a função de perda definida (`criterion`, que neste caso é `CrossEntropyLoss`).

#### **(e) Backward pass**
```python
loss.backward()
```
- Calcula os gradientes da perda em relação aos pesos do modelo usando o algoritmo de backpropagation. Esses gradientes são armazenados nos parâmetros do modelo (acessíveis via `model.parameters()`).

#### **(f) Atualização dos pesos**
```python
optimizer.step()
```
- Atualiza os pesos do modelo usando os gradientes calculados no passo anterior. O otimizador (`optimizer`, que neste caso é `Adam`) aplica a regra de atualização apropriada com base nos gradientes e na taxa de aprendizado (`learning_rate`).

#### **(g) Acumulação da perda**
```python
total_loss += loss.item()
```
- Adiciona a perda do lote atual (`loss.item()`) ao total acumulado da época (`total_loss`). O método `.item()` é usado para obter o valor escalar da perda como um número Python.

---

### **4. Exibição da perda média por época**
```python
print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}')
```
- Após o término de todos os lotes em uma época, calcula-se a perda média dividindo o total acumulado (`total_loss`) pelo número de lotes (`len(train_loader)`).
- Exibe a perda média para a época atual no formato especificado.

---

### **Resumo do Processo**
1. **Forward pass**: Os dados de entrada são passados pelo modelo para gerar previsões.
2. **Cálculo da perda**: A diferença entre as previsões e os rótulos verdadeiros é avaliada usando a função de perda.
3. **Backward pass**: Os gradientes da perda em relação aos pesos do modelo são calculados.
4. **Atualização dos pesos**: Os pesos do modelo são atualizados pelo otimizador com base nos gradientes.
5. **Monitoramento**: A perda média por época é calculada e exibida para acompanhar o progresso do treinamento.

---

### **Importância do Loop de Treinamento**
O loop de treinamento é o coração do aprendizado em redes neurais. Ele ajusta os pesos do modelo para minimizar a função de perda e, consequentemente, melhorar o desempenho do modelo no conjunto de dados de treinamento. O objetivo final é que o modelo generalize bem para novos dados (conjunto de teste ou validação).

## Avaliação no conjunto de teste